In [0]:
import os

!sudo python -m pip install -q google-cloud
!pip install -q --upgrade google-api-python-client
!pip install -q --upgrade google-cloud-storage

if os.path.exists('trec_eval') == False:
  !git clone https://github.com/usnistgov/trec_eval.git
  !cd trec_eval; make quicktest

#!pip install -q tensorflow==1.12.0
# tensorflow 1.13.0.rc0 could not run the following
#tf_logger = tf_logging._get_logger()

In [4]:
# authenticate & make TPU ready
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import logging
import pdb

from datetime import timedelta
from google.colab import auth
from google.cloud import storage
from google.cloud.storage import Bucket
from tensorflow.python.platform import tf_logging

try:
  tf.app.flags.DEFINE_string('f', '', 'kernel')
  # (fix this error?) absl.flags._exceptions.UnrecognizedFlagError: Unknown command line flag 'f'
except:
  pass

TPU_FLAG = True #False #True

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credential.json'

try:
  tf_logger = tf_logging._get_logger()
  formatter = logging.Formatter('%(asctime)s %(message)s')
  tf_logger.handlers[0].setFormatter(formatter)
except:
  print('unable to run: tf_logging._get_logger')
  
#log = logging.getLogger(__name__)
def init_log(run_log='run_log.txt'):
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', 
                        level=logging.INFO)

    fh = logging.FileHandler(run_log, encoding="utf-8") # or latin1
    formatter = logging.Formatter('%(asctime)s %(message)s')
    fh.setFormatter(formatter)
    log.addHandler(fh)
    log.info('init log: done')    

#init_log()

# prepare credential.json
s = r'''
{
  "skipped"
}
'''
fn = '/content/credential.json'
with open(fn, 'w') as f:
  f.write(s)

# authentication for TPU
#!ls /content
!gcloud auth activate-service-account --key-file /content/credential.json

if TPU_FLAG:
  assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)
  os.environ['TPU_ADDRESS'] = TPU_ADDRESS

  with tf.Session(TPU_ADDRESS) as session:
    #print('TPU devices:')
    pprint.pprint(session.list_devices())

    # Upload credentials to TPU.
    with open('/content/credential.json', 'r') as f:   #adc.json
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.
    print('credential for TPU: ready')
  

unable to run: tf_logging._get_logger
Activated service account credentials for: [service-colab@project-colab.iam.gserviceaccount.com]
TPU address is grpc://10.32.31.114:8470
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 9922225831524886335),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2265647070501992636),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 707851205934088896),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5841235866533670834),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2698018763439484418),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 17424564979902666527),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 4258848455919520325),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 

In [5]:
# set CPC_ID, data range, etc.
import os

# mapping between Google "cpc_current" table and official CPC hierarchy
'''
  section_id --> CPC Section --> A
  subsection_id --> CPC Class --> A01
  group_id --> CPC Subclass --> A01B
  subgroup_id --> CPC Group & Main group --> A01B33/00
'''

# default settings
FIXED_NUM_TRAIN_STEPS = None  
CALC_METRICS_V1 = False
CALC_METRICS_V2 = False
#

# case 1-7, train 1 step only
'''
DATA_YEAR = 'all.2014~2006' 
CPC_ID = 'group_id' 
PREFIX = None #'Y' #'H' #'G' #'F' #'E' #'D' #'C' #'B' #'A'
TASK = 'PMLP' # for claims
DATASET_PREFIX = 'claim'
THRESHOLD = 0.3
USE_CLAIM_TAIL = False
FIXED_NUM_TRAIN_STEPS = 1
'''

# case 1-6
'''
DATA_YEAR = '2010'
CPC_ID = 'group_id' 
PREFIX = None #'Y' #'H' #'G' #'F' #'E' #'D' #'C' #'B' #'A'
TASK = 'PMLP' # for claims
DATASET_PREFIX = 'claim'
THRESHOLD = 0.3
USE_CLAIM_TAIL = False
'''

# case 1-5
'''
DATA_YEAR = 'all.2014~2006'
CPC_ID = 'section_id' 
PREFIX = None #'Y' #'H' #'G' #'F' #'E' #'D' #'C' #'B' #'A'
TASK = 'PMLP' # for claims
DATASET_PREFIX = 'claim'
THRESHOLD = 0.3
USE_CLAIM_TAIL = False
'''

# case 1-4
'''
USE_CLAIM_TAIL = True   # worse
DATA_YEAR = 'all.2014~2006' #'all.2014~2000' #'all.2016~2016'
CPC_ID = 'group_id' 
PREFIX = None #'Y' #'H' #'G' #'F' #'E' #'D' #'C' #'B' #'A'
TASK = 'PMLP' # for claims
DATASET_PREFIX = 'claim'
THRESHOLD = 0.3
'''

# case 1-3
'''
DATA_YEAR = 'all.2014~2000' #'all.2014~1999' #'all.2016~2016'
CPC_ID = 'group_id' 
PREFIX = None #'Y' #'H' #'G' #'F' #'E' #'D' #'C' #'B' #'A'
TASK = 'PMLP' # for claims
DATASET_PREFIX = 'claim'
THRESHOLD = 0.3
USE_CLAIM_TAIL = False
'''

# case 1-2  ***** important *****
DATA_YEAR = 'all.2014~2006' #'all.2014~2000' #'all.2016~2016'
CPC_ID = 'group_id' 
PREFIX = None #'Y' #'H' #'G' #'F' #'E' #'D' #'C' #'B' #'A'
TASK = 'PMLP' # for claims
DATASET_PREFIX = 'claim'
THRESHOLD = 0.3
USE_CLAIM_TAIL = False
'''
'''

# case 1-1
'''
DATA_YEAR = 'all.2017~2016'
CPC_ID = 'group_id' 
PREFIX = None #'Y' #'H' #'G' #'F' #'E' #'D' #'C' #'B' #'A'
TASK = 'PMLP' # for claims
DATASET_PREFIX = 'claim'
THRESHOLD = 0.3
USE_CLAIM_TAIL = False
'''

# case 2-2, ipc-based
'''
DATA_YEAR = 'all.2014~2006'   # for IPC
CPC_ID = 'ipc'
PREFIX = None
TASK = 'USPTO-2M-claim' 
DATASET_PREFIX = 'USPTO-2M-claim'
THRESHOLD = 0.3
USE_CLAIM_TAIL = False
'''

# case 2-1, ipc-based
'''
DATA_YEAR = 'all.2014~2006'   # for IPC
CPC_ID = 'ipc'
PREFIX = None
TASK = 'USPTO-2M' # for USPTO-2M, title + abstract
DATASET_PREFIX = 'USPTO-2M-title-abstract'
THRESHOLD = 0.3
USE_CLAIM_TAIL = False
'''

MAX_SEQ_LENGTH = 128  # default

if PREFIX is not None:
  DATA_YEAR += '.%s' % PREFIX

OUTPUT_YEAR = DATA_YEAR # default
if FIXED_NUM_TRAIN_STEPS is not None:
  OUTPUT_YEAR += '.step.%s' % FIXED_NUM_TRAIN_STEPS
  
tf.logging.info('OUTPUT_YEAR: %s' % OUTPUT_YEAR)

#CPC_ID = 'ipc'             # 632 classes
#CPC_ID = 'section_id'      # A~H & Y
#CPC_ID = 'subsection_id'   # A01
#CPC_ID = 'group_id'        # A01B  --> 656 classes
#CPC_ID = 'subgroup_id'     # C40B40/14

#DATA_YEAR = '2016'
#DATA_YEAR = '2017'
#DATA_YEAR = 'all.2017~2000'
#DATA_YEAR = 'all.2017~2012'
#DATA_YEAR = 'all.2017~2013'
#DATA_YEAR = 'all.2017~2014'
#DATA_YEAR = 'all.2017~2015'
#DATA_YEAR = 'all.2017~2016'
#OUTPUT_YEAR = 'all.2017~2014'
#OUTPUT_YEAR = DATA_YEAR + '.MAX_SEQ_LENGTH.3'
#OUTPUT_YEAR = DATA_YEAR + '.MAX_SEQ_LENGTH.10'
#OUTPUT_YEAR = DATA_YEAR + '.step.1'
#OUTPUT_YEAR = DATA_YEAR + '.times.3'
#MAX_SEQ_LENGTH = 3
#MAX_SEQ_LENGTH = 10 
#MAX_SEQ_LENGTH = 256
#MAX_SEQ_LENGTH = 512
#


#TASK = 'PMLP' # for claims
#DATASET_PREFIX = 'claim'

#TASK = 'USPTO-2M' # for USPTO-2M, title + abstract
#DATASET_PREFIX = 'USPTO-2M-title-abstract'

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model

BERT_MODEL = 'uncased_L-12_H-768_A-12' 
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR
os.environ['BERT_PRETRAINED_DIR'] = BERT_PRETRAINED_DIR
!echo $BERT_PRETRAINED_DIR

BUCKET = 'patent_bert' 
TASK_DATA_DIR = 'gs://{}/patents-public-data/{}_{}.{}/'.format(BUCKET, 
                DATASET_PREFIX, CPC_ID, DATA_YEAR) 
OUTPUT_DIR = 'gs://{}/models/{}.{}.{}'.format(BUCKET, 
                                              TASK, CPC_ID, OUTPUT_YEAR)
if USE_CLAIM_TAIL:
  OUTPUT_DIR += '.tail'
  
#OUTPUT_DIR = 'gs://{}/models/{}.{}'.format(BUCKET, TASK, OUTPUT_YEAR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
if TASK == 'PMLP' or TASK.find('USPTO') >= 0:
  !gsutil ls $TASK_DATA_DIR
else:
  !ls $TASK_DATA_DIR

#pdb.set_trace()

INFO:tensorflow:OUTPUT_YEAR: all.2014~2006
***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12
***** Model output directory: gs://patent_bert/models/PMLP.group_id.all.2014~2006 *****
***** Task data directory: gs://patent_bert/patents-public-data/claim_group_id.all.2014~2006/ *****
gs://patent_bert/patents-public-data/claim_group_id.all.2014~2006/
gs://patent_bert/patents-public-data/claim_group_id.all.2014~2006/(ipc) USPTO-2M-claim_2015~2015.t

In [6]:
# ready to run --> run_classifier_cpc
# set FLAGS by my own
# Setup task specific model and TPU running config.

import modeling
import optimization
import run_classifier_cpc 
import tokenization
import pandas as pd

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0  # 4.0 #20.0 #3.0
WARMUP_PROPORTION = 0.1

# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

flags = tf.flags
FLAGS = flags.FLAGS

#J
FLAGS.do_train = True #False #True  
FLAGS.do_eval = False #False # True
FLAGS.do_predict = True # True
FLAGS.cpc_id = CPC_ID
FLAGS.prefix = PREFIX
FLAGS.file_based_flag = True
FLAGS.reuse_tf_record = True # True
FLAGS.replace_eval_with_predict_tf_record = False
FLAGS.multi_hot_threshold = THRESHOLD # = 0.3 #0.3 #0.5 #0.3 #50  # top 50 
FLAGS.use_claim_tail = USE_CLAIM_TAIL
FLAGS.fixed_num_train_steps = FIXED_NUM_TRAIN_STEPS
FLAGS.calc_metrics_v1 = CALC_METRICS_V1
FLAGS.calc_metrics_v2 = CALC_METRICS_V2

# not necessary 
#INIT_CHECKPOINT = os.path.join(OUTPUT_DIR, 'model.ckpt-94000')
#INIT_CHECKPOINT = os.path.join(OUTPUT_DIR, 'model.ckpt-102000')
#

# original: set flags
FLAGS.train_batch_size = TRAIN_BATCH_SIZE
FLAGS.eval_batch_size = EVAL_BATCH_SIZE
FLAGS.learning_rate = LEARNING_RATE
FLAGS.num_train_epochs = NUM_TRAIN_EPOCHS
FLAGS.warmup_proportion = WARMUP_PROPORTION
FLAGS.max_seq_length = MAX_SEQ_LENGTH

FLAGS.use_tpu = TPU_FLAG  #False #True
if TPU_FLAG:
  FLAGS.tpu_name = TPU_ADDRESS
else:
  FLAGS.tpu_name = ''
FLAGS.data_dir = TASK_DATA_DIR
FLAGS.output_dir = OUTPUT_DIR
FLAGS.vocab_file = VOCAB_FILE
FLAGS.bert_config_file = CONFIG_FILE
FLAGS.init_checkpoint = INIT_CHECKPOINT
FLAGS.do_lower_case = DO_LOWER_CASE

run_classifier_cpc.main('')


INFO:tensorflow:Existed: gs://patent_bert/models/PMLP.group_id.all.2014~2006 
INFO:tensorflow:labels: ['A01B', 'A01C', 'A01D', 'A01F', 'A01G', 'A01H', 'A01J', 'A01K', 'A01L', 'A01M', 'A01N', 'A21B', 'A21C', 'A21D', 'A22B', 'A22C', 'A23B', 'A23C', 'A23D', 'A23F', 'A23G', 'A23J', 'A23K', 'A23L', 'A23N', 'A23P', 'A23V', 'A23Y', 'A24B', 'A24C', 'A24D', 'A24F', 'A41B', 'A41C', 'A41D', 'A41F', 'A41G', 'A41H', 'A42B', 'A42C', 'A43B', 'A43C', 'A43D', 'A44B', 'A44C', 'A44D', 'A45B', 'A45C', 'A45D', 'A45F', 'A46B', 'A46D', 'A47B', 'A47C', 'A47D', 'A47F', 'A47G', 'A47H', 'A47J', 'A47K', 'A47L', 'A61B', 'A61C', 'A61D', 'A61F', 'A61G', 'A61H', 'A61J', 'A61K', 'A61L', 'A61M', 'A61N', 'A61Q', 'A62B', 'A62C', 'A62D', 'A63B', 'A63C', 'A63D', 'A63F', 'A63G', 'A63H', 'A63J', 'A63K', 'B01B', 'B01D', 'B01F', 'B01J', 'B01L', 'B02B', 'B02C', 'B03B', 'B03C', 'B03D', 'B04B', 'B04C', 'B05B', 'B05C', 'B05D', 'B06B', 'B07B', 'B07C', 'B08B', 'B09B', 'B09C', 'B21B', 'B21C', 'B21D', 'B21F', 'B21G', 'B21H', 'B21J', '

Exception ignored in: <generator object TPUEstimator.predict at 0x7f214f7f3eb8>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/tpu_estimator.py", line 2500, in predict
    rendezvous.raise_errors()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/error_handling.py", line 132, in raise_errors
    six.reraise(typ, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/six.py", line 693, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/error_handling.py", line 101, in catch_errors
    yield
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/tpu_estimator.py", line 478, in _run_infeed
    session.run(self._enqueue_ops)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 929, in run
    run_metadata_ptr)
  File "/usr/local/lib/python3.6/dist-packages/tensor

In [7]:
#!head -3 v2_trec_qrels.txt
#!wc v2_trec_qrels.txt
!trec_eval/trec_eval -m P.1,2,3,4,5 -m recall.1,2,3,4,5,50,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20 v2_trec_qrels.txt v2_trec_results.txt
#!trec_eval/trec_eval v2_trec_qrels.txt v2_trec_results.txt
#paused = input('paused....')
sys.exit(1)

tcmalloc: large alloc 1386627072 bytes == 0x5644fb6e2000 @  0x7fe1d156b1e7 0x5644f9cce023 0x5644f9ccb2a8 0x7fe1d0dcab97 0x5644f9ccad9a
tcmalloc: large alloc 2361622528 bytes == 0x56454e146000 @  0x7fe1d156b1e7 0x5644f9cce147 0x5644f9ccb2a8 0x7fe1d0dcab97 0x5644f9ccad9a
tcmalloc: large alloc 2361622528 bytes == 0x5645dad7e000 @  0x7fe1d156b1e7 0x7fe1d0deb5c0 0x5644f9cce23c 0x5644f9ccb2a8 0x7fe1d0dcab97 0x5644f9ccad9a
tcmalloc: large alloc 1574412288 bytes == 0x5645db45e000 @  0x7fe1d156b1e7 0x5644f9cce317 0x5644f9ccb2a8 0x7fe1d0dcab97 0x5644f9ccad9a
tcmalloc: large alloc 2862637056 bytes == 0x5646391d8000 @  0x7fe1d156b1e7 0x5644f9cce9d1 0x5644f9ccb39f 0x7fe1d0dcab97 0x5644f9ccad9a
tcmalloc: large alloc 2361622528 bytes == 0x56454e146000 @  0x7fe1d156b1e7 0x5644f9cceb14 0x5644f9ccb39f 0x7fe1d0dcab97 0x5644f9ccad9a
tcmalloc: large alloc 2361622528 bytes == 0x564712c9e000 @  0x7fe1d156b1e7 0x7fe1d0deb5c0 0x5644f9ccec76 0x5644f9ccb39f 0x7fe1d0dcab97 0x5644f9ccad9a
P_1                   	al

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
